[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RichmondAlake/memorizz/blob/main/examples/knowledge_base.ipynb)

In [1]:
! pip install -qU memorizz

In [2]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [3]:

set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [4]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API Key: ")

In [5]:
set_env_securely("VOYAGE_API_KEY", "Enter your VOYAGE_API_KEY API Key: ")

In [6]:
from memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a mongodb config with voyageai embeddings
mongodb_config = MongoDBConfig(
    uri=os.environ["MONGODB_URI"],
    db_name="testing_memorizz",
    embedding_provider="voyageai",
    embedding_config={
        "embedding_type": "contextualized",
        "model": "voyage-context-3",
        "output_dimension": 256,
    }
)

# Create a memory provider
memory_provider = MongoDBProvider(mongodb_config)

/Users/richmondalake/miniconda3/envs/agent_memory_course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector search index 'vector_index' already exists for collection personas.
Vector search index 'vector_index' already exists for collection toolbox.
Vector search index 'vector_index' already exists for collection short_term_memory.
Vector search index 'vector_index' already exists for collection long_term_memory.
Vector search index 'vector_index' already exists for collection conversation_memory.
Vector search index 'vector_index' already exists for collection workflow_memory.
Vector search index 'vector_index' already exists for collection agents.
Vector search index 'vector_index' already exists for collection shared_memory.
Vector search index 'vector_index' already exists for collection summaries.


In [7]:
from memorizz.long_term_memory import KnowledgeBase

# Create a knowledge base
kb = KnowledgeBase(memory_provider)

In [8]:
# Sample knowledge to ingest
company_info = """
Acme Corporation is a fictional company that manufactures everything from portable holes to rocket-powered devices.
Founded in 1952, Acme has been a leading supplier of innovative products for over 70 years.
The company is headquartered in California and has 5,000 employees worldwide.
Their most popular products include the Portable Hole, Rocket Skates, and Giant Rubber Band.
"""

product_info = """
Acme's Portable Hole is a revolutionary product that creates a temporary hole in any surface.
Simply place the Portable Hole on a wall, floor, or ceiling, and it creates a passage to the other side.
The hole can be folded up and carried in a pocket when not in use.
Warning: Do not stack Portable Holes or place them face-to-face, as this may create a rift in space-time.
"""

In [9]:
# Ingest the knowledge
print("Ingesting knowledge...")
company_memory_id = kb.ingest_knowledge(company_info, namespace="company_info")
product_memory_id = kb.ingest_knowledge(product_info, namespace="product_info")
print(f"Created knowledge entries with IDs: {company_memory_id}, {product_memory_id}")

Ingesting knowledge...
Created knowledge entries with IDs: 689caabdb3de2baa6380a973, 689caabdb3de2baa6380a975


In [10]:
from memorizz import MemAgent

# Create a simple agent
agent = MemAgent(
    instruction="You are a helpful assistant that provides information about Acme Corporation and its products.",
    memory_provider=memory_provider
)

In [11]:
# Save the agent to get an agent_id
agent.save()
print(f"Created agent with ID: {agent.agent_id}")


Created agent with ID: 689caabeb3de2baa6380a977


In [12]:
# Method 1: Attach knowledge using the KnowledgeBase
kb.attach_to_agent(agent, company_memory_id)
print("Attached company info to agent")

Attached company info to agent


In [13]:
# Method 2: Attach knowledge using the agent's method
agent.add_long_term_memory(product_info, namespace="additional_product_info")
print("Added product info via agent method")

Added product info via agent method


In [14]:
# Retrieve and print all long-term memories associated with the agent
memories = agent.retrieve_long_term_memory()
print(f"\nAgent has {len(memories)} knowledge entries:")
for i, memory in enumerate(memories, 1):
    print(f"{i}. Namespace: {memory.get('namespace')}")
    print(f"   Content: {memory.get('content')[:50]}...")


Agent has 2 knowledge entries:
1. Namespace: company_info
   Content: 
Acme Corporation is a fictional company that manu...
2. Namespace: additional_product_info
   Content: 
Acme's Portable Hole is a revolutionary product t...


In [15]:
# Run some queries to demonstrate knowledge retrieval
print("\nRunning queries with the agent:")

queries = [
    "What is Acme Corporation?",
    "Tell me about the Portable Hole.",
    "When was Acme founded?"
]

for query in queries:
    print(f"\nQuery: {query}")
    response = agent.run(query)
    print(f"Response: {response}")


Running queries with the agent:

Query: What is Acme Corporation?
Response: Acme Corporation is a fictional company, commonly known for its wide variety of inventive and outlandish products. Founded in 1952 and headquartered in California, Acme has established itself (in the realms of cartoons and fiction) as a leading supplier of creative and often humorous devices. With about 5,000 employees worldwide, the company is recognized for popular products such as the Portable Hole, Rocket Skates, and the Giant Rubber Band.

Acme’s products are famously featured in classic animation—particularly the Looney Tunes series—where they are frequently used (and misused) by characters like Wile E. Coyote. Over more than 70 years, Acme has become synonymous with imaginative—and sometimes unpredictable—gadgets that capture the spirit of invention.

If you want more details about Acme’s history or specific products, let me know!

Query: Tell me about the Portable Hole.
Response: The Acme Portable Hole

In [16]:
# Clean up (optional)
print("\nCleaning up...")
agent.delete_long_term_memory(company_memory_id)
agent.delete()


Cleaning up...


True